In [1]:
import matplotlib.pyplot as plt

C:\Users\SWAPNIL\AppData\Roaming\Python\Python39\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers

In [3]:
Width=135
Height=85
BATCH_SIZE=64
CHANNELS=3
EPOCHS=30

dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "Oil Ageing",
    shuffle=True,
    image_size=(Width,Height),
    batch_size=BATCH_SIZE
)

Found 5858 files belonging to 4 classes.


In [4]:
class_names=dataset.class_names
class_names

['Fresh', 'Highly Aged', 'Lightly Aged', 'Moderately Aged']

In [5]:
def get_datset_partitions_tf(ds,train_split=0.7,val_split=0.15,shuffle=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)
    train_ds=ds.take(train_size)
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    return train_ds,val_ds,test_ds

In [6]:
train_ds,val_ds,test_ds=get_datset_partitions_tf(dataset)

In [7]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
TEST_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [8]:
resize_and_rescale=tf.keras.Sequential ([
    layers.Resizing(256,256),
    layers.Rescaling(1.0/255)
])

In [9]:
n_classes=4 
INPUT_SHAPE=(BATCH_SIZE,256,256,CHANNELS)
model=models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=INPUT_SHAPE),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
])

model.build(input_shape=INPUT_SHAPE)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (64, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (64, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (64, 127, 127, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (64, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (64, 62, 62, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (64, 60, 60, 64)         

In [11]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history=model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/30


64/64 [==============================] - 119s 2s/step - loss: 0.8224 - accuracy: 0.6067 - val_loss: 0.5696 - val_accuracy: 0.7019
Epoch 2/30
64/64 [==============================] - 131s 2s/step - loss: 0.4312 - accuracy: 0.8151 - val_loss: 0.2633 - val_accuracy: 0.9038
Epoch 3/30
64/64 [==============================] - 109s 2s/step - loss: 0.2753 - accuracy: 0.8817 - val_loss: 0.1674 - val_accuracy: 0.9435
Epoch 4/30
64/64 [==============================] - 131s 2s/step - loss: 0.2055 - accuracy: 0.9147 - val_loss: 0.2608 - val_accuracy: 0.8846
Epoch 5/30
64/64 [==============================] - 119s 2s/step - loss: 0.1032 - accuracy: 0.9611 - val_loss: 0.0599 - val_accuracy: 0.9820
Epoch 6/30
64/64 [==============================] - 115s 2s/step - loss: 0.0451 - accuracy: 0.9855 - val_loss: 0.0901 - val_accuracy: 0.9688
Epoch 7/30
64/64 [==============================] - 111s 2s/step - loss: 0.1135 - accuracy: 0.9567 - val_loss: 0.0382 - val_accuracy: 0.9856
Epoch 8/30


In [ ]:
history.history.keys()

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label='Training Accuracy')
plt.plot(range(EPOCHS),val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label='Training Loss')
plt.plot(range(EPOCHS),val_loss,label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import numpy as np
for images_batch,labels_batch in test_ds.take(1):
    first_image=images_batch[0].numpy().astype('uint8')
    first_label=labels_batch[0]
    
    print("First image to predict")
    plt.imshow(first_image)
    print("First image's actual label : ",class_names[first_label])
    
    batch_prediction=model.predict(images_batch)
    print(np.argmax(batch_prediction[0]))

In [ ]:
def predict(model,img):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0)
    
    predictions=model.predict(img_array)
    
    predicted_class=class_names[np.argmax(predictions[0])]
    confidence=round(100*(np.max(predictions[0])),2)
    return predicted_class,confidence

In [ ]:
plt.figure(figsize=(15,15))
for images,labels in test_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        
        predicted_class,confidence=predict(model,images[i].numpy())
        actual_class=class_names[labels[i]]
        
        plt.title(f"Actual : {actual_class},\n Predicted : {predicted_class} , \n Confidence : {confidence} % ")
        plt.axis('off')

In [ ]:
import h5py

In [ ]:
import os
model.save(f"../Models/1.keras")